In [2]:
import sys
import os
import shutil
import glob
import random
import math
from numpy import random
import numpy as np
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy
import pandas
import sklearn
from collections import Counter
from matplotlib.font_manager import FontProperties
from matplotlib import ticker
from pymoo.core.problem import ElementwiseProblem
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.decomposition.asf import ASF
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.termination import get_termination
from pymoo.decomposition.asf import ASF
from pymoo.optimize import minimize
import matplotlib.patches as mpatches
import time
from matplotlib.pyplot import MultipleLocator
from itertools import product
font = FontProperties(fname=r"c:\windows\fonts\simhei.ttf", size=14)
fontlegend = FontProperties(fname=r"c:\windows\fonts\simhei.ttf", size=14)
print(sys.version)
print('numba version: {}'.format(np.__version__))
print('flopy version: {}'.format(flopy.__version__))

3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
numba version: 1.23.4
flopy version: 3.3.4


In [3]:
timel=[]
with open('D:\\工作需\\R程序计算\\R程序翻译\\timeC13.txt', 'r+') as obj: #time file path
    for line in obj.readlines():
        timel.append(line)
    timel = [line.strip("\n") for line in timel]
Tlist=np.array(timel)

In [1]:
'''
read mppth file and save IJK
'''
def save_mp_resnb(pth_file, out_csv, tar_colnames=None, add_ij_info=False, in_model=None):
    if tar_colnames == None:
        tar_colnames = ["particleid", "x", "y", "z","time", "k"]
    pthobj = flopy.utils.PathlineFile(pth_file)
    df_all = pandas.DataFrame(pthobj._data)[tar_colnames]
    if add_ij_info:
        i,j = in_model.dis.get_rc_from_node_coordinates(df_all["x"].values, df_all["y"].values)
        df_all.insert(len(df_all.columns), "i", i)
        df_all.insert(len(df_all.columns), "j", j)
    df_all.to_csv(out_csv, index=None)
    return "Complete: %s"%(pth_file)

# CELL.txt

In [17]:
workspace = os.path.join('.')
def mpthcell(allname, modelname, welnum, hk):
    nm=modelname
    is_silent = True
    in_mfn='D:/工作需/G-case/%s_MODFLOW_text/%s.mfn'%(allname,allname) #C13-test_MODFLOW_text path
    m = flopy.modflow.Modflow.load(in_mfn, model_ws=workspace, version="mf2005", exe_name = 'mf2005.exe')
    nrow, ncol, nlay, nper = m.nrow_ncol_nlay_nper
    m.write_input()
    m.run_model(silent=is_silent)
    
    #particle in each cell
    plocs = []
    pids = []
    ids=0
    for kdx, idx, jdx in product(range(nlay),range(nrow),range(ncol)):
        plocs.append((kdx, idx, jdx))
        ids+=1
        pids.append(ids)
    part0 = flopy.modpath.ParticleData(
        plocs, drape=0, structured=True, particleids=pids
    )
    pg0 = flopy.modpath.ParticleGroup(
        particlegroupname="PG1", particledata=part0, filename="ex01a.pg1.sloc"
    )
    particlegroups = [pg0]
    import time
    for idx,i in enumerate(Tlist):
        s = time.time()
        # create modpath files
        exe_name = "mp7"
        mp = flopy.modpath.Modpath7(
            modelname=nm + "_mp", flowmodel=m, exe_name=exe_name, model_ws=workspace, budgetfilename="%s.ccf"%allname
        )
        mpbas = flopy.modpath.Modpath7Bas(mp)
        mpsim = flopy.modpath.Modpath7Sim(
            mp,
            simulationtype="pathline",
            trackingdirection="forward",
            weaksinkoption="pass_through",
            weaksourceoption="pass_through",
            budgetoutputoption="summary",
            referencetime=[0, 0, 0.0],
            stoptimeoption="specified",
            stoptime=int(i)-1,
            zonedataoption="off",
            particlegroups=particlegroups,
        )

        # write modpath datasets
        mp.write_input()
        # run modpath
        mp.run_model(silent=True)
        fpth_file = workspace+"/"+nm+"_mp.mppth"
        out_csv = workspace+"/modpath_midresult.csv"
        save_mp_resnb(fpth_file, out_csv, add_ij_info=True, in_model=m)

        df=pandas.read_csv(out_csv)
        c=df.values.tolist()
        carr=np.array(c, dtype='object')
        rows, columns = carr.shape
        columns=['Particle_Index', 'X', 'Y', 'Z', 'Time', 'Cell_K', 'Cell_I', 'Cell_J']
        for i,j in product(range(rows),range(5,8)): 
            carr[i,j]=int(carr[i,j]+1)
            carr[i,0]=int(carr[i,0])
        carr=np.insert(carr, 0, values=columns, axis=0) 
        '''save path'''
        txtc=[]
        for i in Tlist:
            txtcell = os.path.join(r"D:\工作需\R程序计算\R程序翻译\%s"%nm, r"cell_%s.txt"%i) #path to save cell.txt
            txtc.append(txtcell)
        wstxt=r"D:/工作需/R程序计算/R程序翻译/"+nm
        if not os.path.exists(wstxt):
            os.makedirs(wstxt)
        with open(txtc[idx], 'w') as f: #保存至指定文件
            np.savetxt(f, carr, fmt="%s", delimiter="    ")
        e = time.time()
        print("%s/%s | %s completed in %.2fs"%(idx+1, len(Tlist), txtc[idx], e-s))
mpthcell(allname='C13-test', modelname='C13-test', welnum=1, hk=0.05)

1/1 | D:\工作需\R程序计算\R程序翻译\C13-test\cell_1800.txt completed in 287.56s


# IN.txt

In [4]:
def mpthin(ptcl,i,inname):
    workspace = os.path.join('.')
    #定义模型名称和存储文件名
    nm='C13-test'
    is_silent = True  
    in_mfn=r'D:/工作需/G-case/C13-test_MODFLOW_text/C13-test.mfn'#C13-test_MODFLOW_text path
    m = flopy.modflow.Modflow.load(in_mfn, model_ws=workspace, version="mf2005", exe_name = 'mf2005.exe')
    nrow, ncol, nlay, nper = m.nrow_ncol_nlay_nper
    m.write_input()
    m.run_model(silent=is_silent)
    #Add particles to all injection wells
    data = pandas.read_table(r"D:\工作需\R程序计算\R程序翻译\井坐标文件\注井坐标C13.txt", sep="\\s+").values.tolist()#well file path
    In_wel=np.array(data)
    inw=In_wel[i:i+2,:]
    plocs = []
    pids = []
    for i in inw:
        plocs.append((int(i[2])-1,int(i[0])-1,int(i[1])-1))
    Plocs=np.concatenate([plocs, plocs], axis=1)
    cd = flopy.modpath.CellDataType(drape=0,
                                   columncelldivisions=ptcl[0],
                                   rowcelldivisions=ptcl[1],
                                   layercelldivisions=ptcl[2])
    p = flopy.modpath.LRCParticleData([cd], [Plocs])

    pg1 = flopy.modpath.ParticleGroupLRCTemplate(particlegroupname="PG2",
                                            particledata=p,
                                            filename="ex01a.pg2.sloc")
    particlegroupin = [pg1] 
    for idx,i in enumerate(Tlist):
        s = time.time()
        # create modpath files
        exe_name = "mp7"
        mp = flopy.modpath.Modpath7(
            modelname=nm + "_mp", flowmodel=m, exe_name=exe_name, model_ws=workspace, budgetfilename="C13-test.ccf"
        )
        mpbas = flopy.modpath.Modpath7Bas(mp)
        mpsim = flopy.modpath.Modpath7Sim(
            mp,
            simulationtype="pathline",
            trackingdirection="forward",
            weaksinkoption="pass_through",
            weaksourceoption="pass_through",
            budgetoutputoption="summary",
            referencetime=[0, 0, 0.0], 
            stoptimeoption="specified",
            stoptime=int(i)-1,
            zonedataoption="off",
            particlegroups=particlegroupin,
        )

        # write modpath datasets
        mp.write_input()
        # run modpath
        mp.run_model(silent=True)
        fpth_file = workspace+"/"+nm+"_mp.mppth"
        out_csv = workspace+"/modpath_midresult.csv"
        save_mp_resnb(fpth_file, out_csv, add_ij_info=True, in_model=m)
        df=pandas.read_csv(out_csv) 
        c=df.values.tolist()
        carr=np.array(c, dtype='object')
        rows, columns = carr.shape
        columns=['Particle_Index', 'X', 'Y', 'Z', 'Time', 'Cell_K', 'Cell_I', 'Cell_J']
        for i,j in product(range(rows),range(5,8)): 
            carr[i,j]=int(carr[i,j]+1)
            carr[i,0]=int(carr[i,0])
        carr=np.insert(carr, 0, values=columns, axis=0)
        '''save path'''
        txtc=[]
        for i in Tlist:
            txtcell = os.path.join(r"D:\工作需\R程序计算\R程序翻译\%s"%nm, inname)#path to save cell.txt
            txtc.append(txtcell)
        wstxt=r"D:/工作需/R程序计算/R程序翻译/"+nm
        if not os.path.exists(wstxt):
            os.makedirs(wstxt)
        with open(txtc[idx], 'w') as f: #保存至指定文件
            np.savetxt(f, carr, fmt="%s", delimiter="    ")
        e = time.time()

In [5]:
#injection wells file path
wel = pandas.read_table(r'D:\工作需\R程序计算\R程序翻译\井坐标文件\注井坐标C13.txt', sep="\\s+").values.tolist()
welarr = np.array(wel)
#ore range file path
mine = pandas.read_table(r'D:\工作需\R程序计算\R程序翻译\0222矿层.txt', sep="\\s+").values.tolist()
arrm=np.array(mine)

# calculation of the characteristic volume of in-situ uranium leaching

In [6]:
dir_path=r"D:/工作需/R程序计算/R程序翻译" 
dir_path2=r"D:/工作需/R程序计算/R程序翻译/C13-test" #cel.txt and in,txt path
timezlh=dir_path+"/timeC13.txt" #time file

zlc_dir=dir_path+"/总流场"
dylc_dir=dir_path+"/单元流场"

jzb_dir=dir_path+"/井坐标文件" 

jzb_file_lis=os.listdir(jzb_dir)
if jzb_file_lis[0].startswith("抽井"):
    cj_path=jzb_dir+"/"+jzb_file_lis[0]
    zj_path=jzb_dir+"/"+jzb_file_lis[1]
else:
    cj_path = jzb_dir + "/" + jzb_file_lis[1]
    zj_path = jzb_dir + "/" + jzb_file_lis[0]
    
def intjudge(x):
    if  isinstance(x, int)==True:
        index=-1
    else:
        index=1
    return index
    
def newly_build_dir():
    """ Check whether the save path exists, create it if it does not exist"""
    for i in (zlc_dir,dylc_dir):
        if not os.path.exists(i):
            os.mkdir(i)

def get_file_list(inname):
    """ Get a list of files to process"""
    with open(timezlh,"r",encoding="utf-8") as f:
        l=f.read().strip().split("\n")
    file_list=[(i,dir_path2+"/"+"cell_"+i+".txt",dir_path2+"/"+inname) for i in l]
    return file_list

def get_cj_ijk():
    """ ijk of pumping wells"""
    data=pandas.read_csv(cj_path,sep="\\s+",usecols=["I","J","K"]).values.tolist()
    set_data=set([tuple(i) for i in data]) 
    return set_data

def get_zj_ijk(file_path):
    """ ijk of injection wells"""
    dic={}
    data = pandas.read_table(zj_path, sep="\\s+").values.tolist()
    for i in data:
        dic.setdefault(i[-1],set()).add(tuple(i[:3]))
    for i in dic:
        dic[i]=tuple(dic[i])
    dic1={}
    data = pandas.read_table(file_path, sep="\\s+", usecols=["Particle_Index","Time", "Cell_K", "Cell_I", "Cell_J"])
    data=data[["Particle_Index","Time", "Cell_I", "Cell_J", "Cell_K"]]
    data["ijk"]=data.apply(lambda x:(x["Cell_I"],x["Cell_J"],x["Cell_K"]),axis=1)
    for k,v in dic.items():
        dic1[k]=set()
        all_index=[]
        for i in v:
            p=data[(data["ijk"]==i) & (data["Time"]==0)]
            l=p["Particle_Index"].values.tolist()
            all_index+=l
        data1=data[data["Particle_Index"].isin(all_index)][["Cell_I", "Cell_J", "Cell_K"]].values.tolist()
        for i in data1:
            dic1[k].add(tuple(i))
    return dic1

def get_set_A(file_path):
    """ get setA"""
    cj_ijk=get_cj_ijk()
    data = pandas.read_table(file_path,sep="\\s+", usecols=["Particle_Index","Time","Cell_K","Cell_I","Cell_J"])
    data["ijk"] = data.apply(lambda x: (x["Cell_I"], x["Cell_J"], x["Cell_K"]), axis=1)
    data1=data[data["ijk"].isin(cj_ijk)]["Particle_Index"].values.tolist()
    d=tuple(set(data1))
    data=data[(data["Time"]==0) & (data["Particle_Index"].isin(d))][["Cell_I","Cell_J","Cell_K"]].values.tolist()
    set_data = set([tuple(i) for i in data]) 
    set_A=set_data
    return set_A

def get_set_B(file_path):
    """ get setB"""
    data = pandas.read_table(file_path, sep="\\s+", usecols=["Time", "Cell_K", "Cell_I", "Cell_J"])
    data = data[["Cell_I", "Cell_J", "Cell_K"]].values.tolist()
    set_data = set([tuple(i) for i in data]) 
    return set_data

def save_csv(csv_path,s):
    """Save a collection of flow fields as a csv file """
    data=pandas.DataFrame(data=tuple(s),columns=["Cell_I", "Cell_J", "Cell_K"])
    data.to_csv(csv_path,index=False)

def zlc_save(d,j,b,c,set_A,set_B):
    """ Save flow field calculation results"""
    d=zlc_dir+"/"+d
    if not os.path.exists(d):
        os.mkdir(d)
    save_csv(d+"/有效对流流场.csv",j)
    save_csv(d+"/总对流流场.csv",b)
    save_csv(d+"/无效对流流场.csv",c)
    save_csv(d+"/set_A.csv",set_A)
    save_csv(d+"/set_B.csv",set_B)
def cal_Lv(data_mine,carr):
    Vsum=0
    data_mine=data_mine.tolist()
    set_mine=set([tuple(i) for i in data_mine])
    Vsum_set=set_mine & carr
    Vsum=len(Vsum_set)*62.109
    return Vsum

def calculation_dylc(zj_ijk,set_A,n):
    """ Calculation unit flow field"""
    save_dic=dylc_dir+"/"+n
    if not os.path.exists(save_dic):
        os.mkdir(save_dic)
    for k,v in zj_ijk.items():
        d=save_dic+"/"+k
        if not os.path.exists(d):
            os.mkdir(d)
        set_jiao = set_A & v  # 交集
        set_bing = set_A | v  # 并集
        set_cha = (set_A - v) | (v-set_A)  # 差集
        save_csv(d + "/有效对流流场.csv", set_jiao)
        save_csv(d + "/总对流流场.csv", set_bing)
        save_csv(d + "/无效对流流场.csv", set_cha)

# single objective optimization

In [16]:
def calculation(ptcl,i, inname):
    newly_build_dir()
    mpthin(ptcl,i,inname)
    for i in get_file_list(inname):
        set_A=get_set_A(i[1])
        set_B=get_set_B(i[2])
        set_jiao=set_A & set_B 
        Vm=cal_Lv(arrm,set_jiao)
        set_bing=set_A | set_B 
        set_cha=(set_A - set_B) | (set_B - set_A) 
        zlc_save(i[0],set_jiao,set_bing,set_cha,set_A,set_B)
        volume=len(set_jiao)*62.109
    return Vm
timearr=[]
for k in range(1,31):
    s=time.time()
    Vm_save=[]
    for i in range(1,21):
        ptcl=[10,10,i]#K direction
        filename='in_1800_%d.txt'%k
        v_tp=calculation(ptcl,2*k-2,filename)#Optimize the number of particles dropped into each injection well separately
        Vm_save.append(v_tp)
    e=time.time()
    timearr.append(e-s)
    with open(r'D:\工作需\R程序计算\R程序翻译\单目标递增\单目标-仅K变-%d.txt'%k, 'w') as f:
        np.savetxt(f, Vm_save, fmt="%s", delimiter="    ")
#save running time
with open(r'D:\工作需\R程序计算\R程序翻译\单目标递增\单目标-仅K变-时间.txt', 'w') as f:
    np.savetxt(f, timearr, fmt="%s", delimiter="    ")

In [18]:
'''read and analyze'''
result=[]
for i in range(1,31):
    D_path=r'D:\工作需\R程序计算\R程序翻译\单目标递增\单目标-仅K变-%d.txt'%i
    D = pandas.read_table(D_path, sep="\\s+").values.tolist()
    arrD=np.array(D)
    for k in range(1,len(arrD)):
        percentage_difference = abs(arrD[k] - arrD[k-1]) / arrD[k-1] * 100
        if percentage_difference <= 3:
            result.append((k+5, arrD[k]))
            break
        if k==len(arrD)-1 and percentage_difference > 3:
            result.append((20,arrD[k]))
'''Save the best value'''
with open(r'D:\工作需\R程序计算\R程序翻译\单目标递增\单目标-仅K变稳定值.txt', 'w') as f:
    np.savetxt(f, result, fmt="%s", delimiter="    ")